# Imports

In [70]:
import requests
from bs4 import BeautifulSoup

# API Creation

In [71]:
def get_view_state(session, url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    view_state_input = soup.find('input', {'name': 'jakarta.faces.ViewState'})
    if view_state_input:
        return view_state_input['value']
    else:
        raise ValueError("ViewState not found on the page")

In [72]:
def query_icsd(structure_query, unique_elements_num, space_group_num):
    session = requests.Session()

    query_url = 'https://icsd.fiz-karlsruhe.de/search/basic.xhtml'
    list_url = 'https://icsd.fiz-karlsruhe.de/display/list.xhtml'

    view_state = get_view_state(session, query_url)

    payload = {
        'content_form': 'content_form',
        'content_form:loginId': '',
        'content_form:password': '',
        'content_form:uiSelectContent': 'EXPERIMENTAL_INORGANIC',
        'content_form:uiSimpleSearch:input': '',
        'content_form:uiBibliographyAuthors:input_input': '',
        'content_form:uiBibliographyAuthors:input_hinput': '',
        'content_form:uiBibliographyPublicationYear:input:input': '',
        'content_form:uiBibliographyJournals:input_input': '',
        'content_form:uiBibliographyJournals:input_hinput': '',
        'content_form:uiBibliographyArticles:input_input': '',
        'content_form:uiBibliographyArticles:input_hinput': '',
        'content_form:uiBibliographyDoi:input_input': '',
        'content_form:uiBibliographyDoi:input_hinput': '',
        'content_form:uiChemistrySearchSumForm:input': structure_query,
        'content_form:uiChemistrySearchElCount:input:input': unique_elements_num,
        'content_form:uiCellParameter:input': '',
        'content_form:uiCellVolume:input:input': '',
        'content_form:uiCellGlobalTolerance:input:input': '',
        'content_form:uiSymmetrySpaceGroupSymbol:input_input': '',
        'content_form:uiSymmetrySpaceGroupSymbol:input_hinput': '',
        'content_form:uiSymmetrySpaceGroupNumber:input:input': space_group_num,
        'content_form:uiSymmetryCrystalSystem:input_input': '',
        'content_form:uiSymmetryCentering:input_input': '',
        'content_form:uiPDFNumber:input_input': '',
        'content_form:uiPDFNumber:input_hinput': '',
        'content_form:uiExperimentalTemperature:input:input': '',
        'content_form:uiExperimentalTemperatureUnit:input_input': 'k',
        'content_form:uiCodeCollection:input:input': '',
        'content_form:uiExperimentalPressure:input:input': '',
        'content_form:uiExperimentalPressureUnit:input_input': 'mpa',
        'content_form:btnRunQuery': '',
        'content_form:uiChemistryVisualSearchElCount': '',
        'content_form:uiChemistryVisualSearchUnitCoefficient_input': 'moles',
        'jakarta.faces.ViewState': view_state
    }

    response = session.post(query_url, data=payload)

    if response.status_code != 200:
        return f"Query failed with status code {response.status_code}"

    list_response = session.get(list_url)

    if list_response.status_code == 200:
        return list_response.text
    else:
        return f"List view failed with status code {list_response.status_code}"

In [73]:
res = query_icsd('Ti Co B', 3, 127)

# Scrape Response

In [74]:
res_soup = BeautifulSoup(res, 'html.parser')

In [75]:
template_row_object = {
    'Collection Code': '',
    'HMS': '',
    'Structure Formula': '',
    'Structure Type': '',
    'Reference': '',
    'Cell Parameters': '',
    'Cell Volume': '',
    'Temperature': '',
    'Calculated Density': '',
    'Publication Year': '',
}

In [76]:
res_table = res_soup.find_all(
    "tbody", class_="ui-datatable-data ui-widget-content")
res_objects = res_table[0].find_all("tr")

all_rows = []

for row in res_objects:
    row_object = template_row_object.copy()

    cells = row.find_all("td")

    cell_data = [cell.a.text if cell.a else cell.text for cell in cells]

    if len(cell_data) >= 10:
        row_object['Collection Code'] = cell_data[1]
        row_object['HMS'] = cell_data[2]
        row_object['Structure Formula'] = cell_data[3]
        row_object['Structure Type'] = cell_data[4]
        row_object['Reference'] = cell_data[5]
        row_object['Cell Parameters'] = cell_data[6]
        row_object['Cell Volume'] = cell_data[7]
        row_object['Temperature'] = cell_data[8]
        row_object['Calculated Density'] = cell_data[9]
        row_object['Publication Year'] = cell_data[10]

        all_rows.append(row_object)

In [77]:
for row_obj in all_rows:
    print(row_obj)

{'Collection Code': '433428', 'HMS': 'P 4/m b m', 'Structure Formula': 'Co5.53 Ti2.47 B2', 'Structure Type': 'Ti3Co5B2', 'Reference': 'Zeitschrift fuer Anorganische und Allgemeine Chemie (2017) 643, (21) p. 1551-1556', 'Cell Parameters': '8.39115(16) 8.39115(16) 3.00954(6) 90 90 90', 'Cell Volume': '211.90', 'Temperature': '296.00', 'Calculated Density': '7.30', 'Publication Year': '2017'}
{'Collection Code': '20140', 'HMS': 'P 4/m b m', 'Structure Formula': 'Ti3 Co5 B2', 'Structure Type': 'Ti3Co5B2', 'Reference': 'Zhurnal Strukturnoi Khimii (1971) 12,  p. 458-461', 'Cell Parameters': '8.489(5) 8.489(5) 3.038(3) 90. 90. 90.', 'Cell Volume': '218.93', 'Temperature': '293.00', 'Calculated Density': '6.98', 'Publication Year': '1971'}


# Space Group to Crystal System

In [1]:
def space_group_to_crystal_system(space_group_number):
    space_groups = [
        {'name': 'Triclinic', 'start': 1, 'stop': 2},
        {'name': 'Monoclinic', 'start': 3, 'stop': 15},
        {'name': 'Orthorhombic', 'start': 16, 'stop': 74},
        {'name': 'Tetragonal', 'start': 75, 'stop': 142},
        {'name': 'Trigonal', 'start': 143, 'stop': 167},
        {'name': 'Hexagonal', 'start': 168, 'stop': 194},
        {'name': 'Cubic', 'start': 195, 'stop': 230}
    ]

    for group in space_groups:
        if group['start'] <= space_group_number <= group['stop']:
            print(group['name'])
            return
    print("Number out of range")

In [2]:
space_group_to_crystal_system(50)

Orthorhombic
